In [191]:
import json
import os
from pathlib import Path
import pandas as pd
import math
from tqdm import tqdm

data_path = Path("../data/humaneval-x/")

root, dirs, files = next(os.walk(data_path / "coverage"))
dataset_path = Path(root)


runnable_data = {}
instruction_data = {}
branch_data = {}

for method in dirs:
    root, dirs, files = next(os.walk(dataset_path / method))
    for series in dirs:
        root, dirs, files = next(os.walk(dataset_path / method / series))
        for model in dirs:
            root, dirs, files = next(os.walk(dataset_path / method / series / model))
            
            res_file_path = Path("../data") / "humaneval-x" / "coverage" / method / series / model / "jacoco.jsonl"
            err_file_path = Path("../data") / "humaneval-x" / "coverage" / method / series / model / "error.jsonl"
            
            df = pd.read_json(res_file_path, lines=True)
            err_df = pd.read_json(err_file_path, lines=True)
            runnable_data.setdefault(model, {})
            tot_exec = len(df) + len(err_df)
            if tot_exec == 0:
                runnable_data[model][method] = 0
            else:
                runnable_data[model][method] = round(len(df) / tot_exec, 4)

            instruction_data.setdefault(model, {})
            instruction_coverage = 0
            if not df.empty:
                instruction_coverage = df["instruction_coverage"].mean()
            instruction_data[model][method] = round(instruction_coverage, 4)
            

            branch_data.setdefault(model, {})
            branch_coverage = 0
            if not df.empty:
                branch_coverage = df["branch_coverage"].mean()
            branch_data[model][method] = round(branch_coverage, 4)



runnable_df = pd.DataFrame(runnable_data)
runnable_df.to_csv(data_path /  "coverage_runnable.csv", index_label = 'model')
print("Avg. mean. runnable tests: ", runnable_df.T.median().mean())

instruction_df = pd.DataFrame(instruction_data)
instruction_df.to_csv(data_path / "coverage_instruction.csv", index_label = 'model')
print("Avg. mean. instruction coverage: ", instruction_df.T.median().mean())


branch_df = pd.DataFrame(branch_data)
branch_df.to_csv(data_path / "coverage_branch.csv", index_label = 'model')
print("Avg. mean. branch coverage: ", branch_df.T.median().mean())

Avg. mean. executable tests:  0.23315000000000002
Avg. mean. instruction coverage:  0.98811
Avg. mean. branch coverage:  0.7854599999999999
